# Bernstein-Vazirani algorithm

**Input:** oracle $f:\{0, 1\}^n \rightarrow \{0, 1\}$ where $f(x) = s\cdot x$ for some $s \in \{0, 1\}^n$

**Goal:** find $s$


To make things fair, we will pick a random $s$:

In [ ]:
import numpy as np
from qat.lang.AQASM import *

def random_oracle(nbbits):
    """ Generates a random oracle """
    # picking a random s
    s = np.random.choice([0, 1], size=nbbits)
    
    # the classical oracle (quite simple)
    def classical_oracle(x):
        return s.dot(x) % 2
    
    # the quantum version of the oracle
    quantum_oracle = QRoutine()
    input_wires = quantum_oracle.new_wires(nbbits)
    output_wire = quantum_oracle.new_wires(1)

    for i, bit in enumerate(s):
        if bit:
            CNOT(input_wires[i], output)
    
    return classical_oracle, quantum_oracle, s

This way we have a classical and a quantum version of the same oracle

In [ ]:
co, qo, secret = random_oracle(5)
print("Secret is", secret)


# With classical bits: standard function call
print(co([0, 1, 1, 0, 1]))

# With qubits: we have to work a bit

rout = QRoutine()
qbits = rout.new_wires(5)
output = rout.new_wires(1)
qo(qbits, output)

%qatdisplay rout --svg

## Classical algorithm

The only way to guess $s$ is to compute each of its components iteratively (i.e $n$ calls to $f$):

In [ ]:
def classical_solve(n, oracle):
    return [oracle([1 if j == i else 0 for j in range(n)]) for i in range(n)]

co, qo, secret = random_oracle(5)

print(classical_solve(5, co))
print(secret)


# Quantum algorithm: Bernstein-Vazirani


We recall the algorithm:

**Step 0:** allocating a register $|0^n\rangle$ to store $x$

**Step 1:** creating a uniform superposition $(H^{\otimes n}|\psi \rangle) $

**Step 2:** use the oracle to add a phase $-1$ to each component $|x\rangle$ s.t $f(x) = 1$

**Step 3:** perform a Hadamard transform  $(H^{\otimes n}|\psi \rangle) $

**Step 4:** the final state contains $|s\rangle$, just measure it!


There is a catch in step 2!

hint: $H\cdot NOT \cdot H = Z$ and $Z|1\rangle = - |1\rangle$


In [ ]:
from qat.qpus import get_default_qpu

def quantum_solve(n, oracle):
    prog = Program()
    qbits = prog.qalloc(n)

    
    tmp = prog.qalloc(1)
    X(tmp)
    H(tmp)
    for qbit in qbits:
        H(qbit)
    
    oracle(qbits, tmp)
    
    for qbit in qbits:
        H(qbit)
    circuit = prog.to_circ()
    job = circuit.to_job(qubits=list(range(n)))
    qpu = get_default_qpu()
    result = qpu.submit(job)
    for sample in result:
        return list(map(int, sample.state.value[0]))
print(quantum_solve(5, qo))
print(secret)
 